In [1]:
import torch
from torch.nn import Transformer
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset



# Define dataset class
# class TextDataset(Dataset):
#     def __init__(self, dataset):
#         self.dataset = dataset

#     def __len__(self):
#         return len(self.dataset)

#     def __getitem__(self, idx):
#         return self.dataset[idx]["text"], int(self.dataset[idx]["label"])


class TextDataset(Dataset):
    def __init__(self, dataset):
        self.texts = dataset["text"]
        self.labels = dataset["label"]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.texts[idx], int(self.labels[idx])


# Load IMDb dataset from Hugging Face
dataset: TextDataset = load_dataset("imdb")



Using the latest cached version of the dataset since imdb couldn't be found on the Hugging Face Hub


ValueError: Couldn't find cache for imdb for config 'default'
Available configs in the cache: ['plain_text']

In [117]:

# num_classes = len(set(dataset["train"]["label"]))

# train_dataset = dataset["train"]
# first_100_train_samples = train_dataset[:100]
# text_dataset = TextDataset(dataset=train_dataset)

# # Define DataLoader
# batch_size = 8
# dataloader = DataLoader(text_dataset, batch_size=batch_size, shuffle=True)



AttributeError: 'TextDataset' object has no attribute 'dataset'

In [118]:
num_classes = len(set(dataset["train"]["label"]))

# Select only the first 50 samples from the training dataset
train_dataset = dataset["train"][:50]

# Create the TextDataset using the first 50 samples
text_dataset = TextDataset(dataset=train_dataset)

# Define DataLoader
batch_size = 8
dataloader = DataLoader(text_dataset, batch_size=batch_size, shuffle=True)


AttributeError: 'TextDataset' object has no attribute 'dataset'

In [ ]:


# Define the model
class SimpleTransformer(torch.nn.Module):
    def __init__(self):
        super(SimpleTransformer, self).__init__()
        self.transformer = Transformer(
            d_model=256,  # Decrease hidden dimension
            nhead=4,      # Decrease number of attention heads
            num_encoder_layers=2,  # Decrease number of layers
            num_decoder_layers=2,
        )
        self.fc = torch.nn.Linear(256, num_classes)  # Assuming num_classes is defined somewhere



    def forward(self, src):
        src_mask = self.transformer.generate_square_subsequent_mask(src.size(1)).to(src.device)
        output = self.transformer(src, src_mask)
        output = self.fc(output)
        return output



In [ ]:
model = SimpleTransformer()


/home/ntlpt59/anaconda3/envs/sample/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
dataloader.batch_size

8

In [ ]:

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch in dataloader:
        inputs, targets = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs.view(-1, outputs.size(-1)), targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss}")

# Save the model


KeyError: 0

In [ ]:
torch.save(model.state_dict(), "imdb_model.pt")
print("Model saved.")